In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import gurobipy as gp
from gurobipy import GRB
import itertools
from itertools import combinations
from itertools import permutations
from random import choice
import json
import cvxpy as cp
from tkinter import _flatten
import copy
import time
import scipy.stats as stats

In [2]:
np.random.seed(1)
random.seed(1)

In [3]:
raw_jd_choice = pd.read_excel('data_processing/choices.xlsm')  
offertimes = raw_jd_choice.groupby('clickset')['clicknum'].sum()[raw_jd_choice.clickset.unique()]
#offertimes.to_csv('offertimes.csv')
inc_prod_num = raw_jd_choice['clickset'].value_counts()[raw_jd_choice.clickset.unique()]
#inc_prod_num.to_csv('inc_prod_num.csv')
choice_prob = raw_jd_choice['clicknum'] / raw_jd_choice.groupby('clickset')['clicknum'].transform('sum')
raw_jd_choice['choice prob'] = choice_prob

assortment_info_df = pd.DataFrame({'assortments':raw_jd_choice.clickset.unique(),'offer_times':offertimes,'includ_prod_num':inc_prod_num})

# extended assortments with outside option 
# transfer to list
clickset = raw_jd_choice['clickset']
clickset_list = []
for cset in clickset:
    num_lst = json.loads(cset)
    #clickset_list.append(num_lst+[0])
    clickset_list.append([0]+num_lst)
raw_jd_choice['clickset'] = clickset_list

# all purchase options in each assortment
purchaseset = raw_jd_choice['purchaseset']
purchaseset_list = []
for pset in purchaseset:
    num_lst = json.loads(pset)
    purchaseset_list.append(num_lst)
raw_jd_choice['purchaseset'] = purchaseset_list

purchasenum_list = raw_jd_choice['clicknum'].tolist()
jd_choice=pd.DataFrame({'clickset':clickset_list,'purchaseset':purchaseset_list,'purchasenum':purchasenum_list,'choice prob':choice_prob})

n = 9 # product size top 8 products and outside option
print('there are {} different products'.format(n))
collection = []
for cset in clickset_list:
    if cset not in collection:
        collection.append(cset)
print('there are {} different assortments'.format(len(collection)))

assortment_info_df['assortments'] = collection
choice_collection = np.zeros((n,len(collection)))
global_num = 0
for i in range(len(collection)):
    local_num =0
    while(local_num < inc_prod_num[i]):
        if purchaseset_list[global_num][0] in collection[i]:
            choice_collection[purchaseset_list[global_num][0]][i] = choice_prob[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print('error!')
full_choice_collection = pd.DataFrame(choice_collection)
full_choice_collection.to_csv('instances/full_instances/full_choice.csv')


there are 9 different products
there are 134 different assortments


In [4]:
assortment_info_df_20 = assortment_info_df[(assortment_info_df['offer_times']>=20)]
assortments_20 = []
for i in range(len(assortment_info_df_20['assortments'])):
    assortments_20.append(assortment_info_df_20['assortments'][i])


jd_choice_20 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_20:
        jd_choice_20.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_20 = assortment_info_df_20['assortments']
choice_collection_20 = np.zeros((n,len(collection_20)))

frequency_collection_20 = np.zeros((n,len(collection_20)))

purchaseset_list_20 = jd_choice_20['purchaseset'].tolist()

inc_prod_num_20 = assortment_info_df_20['includ_prod_num']
choice_prob_20 = jd_choice_20['choice prob']
frequency_20 = jd_choice_20['purchasenum']

global_num = 0
for i in range(len(collection_20)):
    local_num =0
    while(local_num < inc_prod_num_20[i]):
        if purchaseset_list_20[global_num][0] in assortments_20[i]:
            choice_collection_20[purchaseset_list_20[global_num][0]][i] = choice_prob_20[global_num]
            frequency_collection_20[purchaseset_list_20[global_num][0]][i] = frequency_20[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_20[global_num])
            print(assortments_20[i])
            print('error!')
            break
#print(global_num)  

choice_check_20 = pd.DataFrame(choice_collection_20)
#choice_check_20.to_csv('instances/full_instances/full_20.csv')
frequency_collection_20 = frequency_collection_20.astype(int)
frequency_check_20 = pd.DataFrame(frequency_collection_20)
#frequency_check_20.to_csv('instances/full_instances/frequency_20.csv')

offer_times_20 = assortment_info_df_20['offer_times'].tolist()
print('the number of assortments with at least 20 purchases is',len(assortments_20))

the number of assortments with at least 20 purchases is 29


In [5]:
assortment_info_df_30 = assortment_info_df[(assortment_info_df['offer_times']>=30)]
assortments_30 = []
for i in range(len(assortment_info_df_30['assortments'])):
    assortments_30.append(assortment_info_df_30['assortments'][i])


jd_choice_30 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_30:
        jd_choice_30.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_30 = assortment_info_df_30['assortments']
choice_collection_30 = np.zeros((n,len(collection_30)))

frequency_collection_30 = np.zeros((n,len(collection_30)))

purchaseset_list_30 = jd_choice_30['purchaseset'].tolist()

inc_prod_num_30 = assortment_info_df_30['includ_prod_num']
choice_prob_30 = jd_choice_30['choice prob']
frequency_30 = jd_choice_30['purchasenum']

global_num = 0
for i in range(len(collection_30)):
    local_num =0
    while(local_num < inc_prod_num_30[i]):
        if purchaseset_list_30[global_num][0] in assortments_30[i]:
            choice_collection_30[purchaseset_list_30[global_num][0]][i] = choice_prob_30[global_num]
            frequency_collection_30[purchaseset_list_30[global_num][0]][i] = frequency_30[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_30[global_num])
            print(assortments_30[i])
            print('error!')
            break
#print(global_num)  

choice_check_30 = pd.DataFrame(choice_collection_30)
#choice_check_30.to_csv('instances/full_instances/full_30.csv')
frequency_collection_30 = frequency_collection_30.astype(int)
frequency_check_30 = pd.DataFrame(frequency_collection_30)
#frequency_check_30.to_csv('instances/full_instances/frequency_30.csv')

offer_times_30 = assortment_info_df_30['offer_times'].tolist()
print('the number of assortments with at least 30 purchases is',len(assortments_30))

the number of assortments with at least 30 purchases is 24


In [6]:
assortment_info_df_40 = assortment_info_df[(assortment_info_df['offer_times']>=40)]
assortments_40 = []
for i in range(len(assortment_info_df_40['assortments'])):
    assortments_40.append(assortment_info_df_40['assortments'][i])


jd_choice_40 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_40:
        jd_choice_40.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_40 = assortment_info_df_40['assortments']
choice_collection_40 = np.zeros((n,len(collection_40)))

frequency_collection_40 = np.zeros((n,len(collection_40)))

purchaseset_list_40 = jd_choice_40['purchaseset'].tolist()

inc_prod_num_40 = assortment_info_df_40['includ_prod_num']
choice_prob_40 = jd_choice_40['choice prob']
frequency_40 = jd_choice_40['purchasenum']

global_num = 0
for i in range(len(collection_40)):
    local_num =0
    while(local_num < inc_prod_num_40[i]):
        if purchaseset_list_40[global_num][0] in assortments_40[i]:
            choice_collection_40[purchaseset_list_40[global_num][0]][i] = choice_prob_40[global_num]
            frequency_collection_40[purchaseset_list_40[global_num][0]][i] = frequency_40[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_40[global_num])
            print(assortments_40[i])
            print('error!')
            break
#print(global_num)  

choice_check_40 = pd.DataFrame(choice_collection_40)
#choice_check_40.to_csv('instances/full_instances/full_40.csv')
frequency_collection_40 = frequency_collection_40.astype(int)
frequency_check_40 = pd.DataFrame(frequency_collection_40)
#frequency_check_40.to_csv('instances/full_instances/frequency_40.csv')

offer_times_40 = assortment_info_df_40['offer_times'].tolist()
print('the number of assortments with at least 40 purchases is',len(assortments_40))

the number of assortments with at least 40 purchases is 19


In [7]:
assortment_info_df_50 = assortment_info_df[(assortment_info_df['offer_times']>=50)]
assortments_50 = []
for i in range(len(assortment_info_df_50['assortments'])):
    assortments_50.append(assortment_info_df_50['assortments'][i])


jd_choice_50 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_50:
        jd_choice_50.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_50 = assortment_info_df_50['assortments']
choice_collection_50 = np.zeros((n,len(collection_50)))

frequency_collection_50 = np.zeros((n,len(collection_50)))

purchaseset_list_50 = jd_choice_50['purchaseset'].tolist()

inc_prod_num_50 = assortment_info_df_50['includ_prod_num']
choice_prob_50 = jd_choice_50['choice prob']
frequency_50 = jd_choice_50['purchasenum']

global_num = 0
for i in range(len(collection_50)):
    local_num =0
    while(local_num < inc_prod_num_50[i]):
        if purchaseset_list_50[global_num][0] in assortments_50[i]:
            choice_collection_50[purchaseset_list_50[global_num][0]][i] = choice_prob_50[global_num]
            frequency_collection_50[purchaseset_list_50[global_num][0]][i] = frequency_50[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_50[global_num])
            print(assortments_50[i])
            print('error!')
            break
#print(global_num)  

choice_check_50 = pd.DataFrame(choice_collection_50)
#choice_check_50.to_csv('instances/full_instances/full_50.csv')
frequency_collection_50 = frequency_collection_50.astype(int)
frequency_check_50 = pd.DataFrame(frequency_collection_50)
#frequency_check_50.to_csv('instances/full_instances/frequency_50.csv')

offer_times_50 = assortment_info_df_50['offer_times'].tolist()
print('the number of assortments with at least 50 purchases is',len(assortments_50))

the number of assortments with at least 50 purchases is 15


In [8]:
assortment_info_df_60 = assortment_info_df[(assortment_info_df['offer_times']>=60)]
assortments_60 = []
for i in range(len(assortment_info_df_60['assortments'])):
    assortments_60.append(assortment_info_df_60['assortments'][i])


jd_choice_60 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_60:
        jd_choice_60.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_60 = assortment_info_df_60['assortments']
choice_collection_60 = np.zeros((n,len(collection_60)))

frequency_collection_60 = np.zeros((n,len(collection_60)))

purchaseset_list_60 = jd_choice_60['purchaseset'].tolist()

inc_prod_num_60 = assortment_info_df_60['includ_prod_num']
choice_prob_60 = jd_choice_60['choice prob']
frequency_60 = jd_choice_60['purchasenum']

global_num = 0
for i in range(len(collection_60)):
    local_num =0
    while(local_num < inc_prod_num_60[i]):
        if purchaseset_list_60[global_num][0] in assortments_60[i]:
            choice_collection_60[purchaseset_list_60[global_num][0]][i] = choice_prob_60[global_num]
            frequency_collection_60[purchaseset_list_60[global_num][0]][i] = frequency_60[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_60[global_num])
            print(assortments_60[i])
            print('error!')
            break
#print(global_num)  

choice_check_60 = pd.DataFrame(choice_collection_60)
#choice_check_60.to_csv('instances/full_instances/full_60.csv')
frequency_collection_60 = frequency_collection_60.astype(int)
frequency_check_60 = pd.DataFrame(frequency_collection_60)
#frequency_check_60.to_csv('instances/full_instances/frequency_60.csv')

offer_times_60 = assortment_info_df_60['offer_times'].tolist()
print('the number of assortments with at least 60 purchases is',len(assortments_60))

the number of assortments with at least 60 purchases is 13


In [9]:
assortment_info_df_75 = assortment_info_df[(assortment_info_df['offer_times']>=75)]
assortments_75 = []
for i in range(len(assortment_info_df_75['assortments'])):
    assortments_75.append(assortment_info_df_75['assortments'][i])


jd_choice_75 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_75:
        jd_choice_75.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_75 = assortment_info_df_75['assortments']
choice_collection_75 = np.zeros((n,len(collection_75)))

frequency_collection_75 = np.zeros((n,len(collection_75)))

purchaseset_list_75 = jd_choice_75['purchaseset'].tolist()

inc_prod_num_75 = assortment_info_df_75['includ_prod_num']
choice_prob_75 = jd_choice_75['choice prob']
frequency_75 = jd_choice_75['purchasenum']

global_num = 0
for i in range(len(collection_75)):
    local_num =0
    while(local_num < inc_prod_num_75[i]):
        if purchaseset_list_75[global_num][0] in assortments_75[i]:
            choice_collection_75[purchaseset_list_75[global_num][0]][i] = choice_prob_75[global_num]
            frequency_collection_75[purchaseset_list_75[global_num][0]][i] = frequency_75[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_75[global_num])
            print(assortments_75[i])
            print('error!')
            break
#print(global_num)  

choice_check_75 = pd.DataFrame(choice_collection_75)
#choice_check_75.to_csv('instances/full_instances/full_75.csv')
frequency_collection_75 = frequency_collection_75.astype(int)
frequency_check_75 = pd.DataFrame(frequency_collection_75)
#frequency_check_75.to_csv('instances/full_instances/frequency_75.csv')

offer_times_75 = assortment_info_df_75['offer_times'].tolist()
print('the number of assortments with at least 75 purchases is',len(assortments_75))

the number of assortments with at least 75 purchases is 12


In [10]:
assortment_info_df_100 = assortment_info_df[(assortment_info_df['offer_times']>=100)]
assortments_100 = []
for i in range(len(assortment_info_df_100['assortments'])):
    assortments_100.append(assortment_info_df_100['assortments'][i])

jd_choice_100 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_100:
        jd_choice_100.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_100 = assortment_info_df_100['assortments']
choice_collection_100 = np.zeros((n,len(collection_100)))

frequency_collection_100 = np.zeros((n,len(collection_100)))

purchaseset_list_100 = jd_choice_100['purchaseset'].tolist()

inc_prod_num_100 = assortment_info_df_100['includ_prod_num']
choice_prob_100 = jd_choice_100['choice prob']
frequency_100 = jd_choice_100['purchasenum']

global_num=0
for i in range(len(collection_100)):
    local_num =0
    while(local_num < inc_prod_num_100[i]):
        if purchaseset_list_100[global_num][0] in assortments_100[i]:
            choice_collection_100[purchaseset_list_100[global_num][0]][i] = choice_prob_100[global_num]
            frequency_collection_100[purchaseset_list_100[global_num][0]][i] = frequency_100[global_num]
            global_num = global_num +1
            local_num = local_num +1

choice_check_100 = pd.DataFrame(choice_collection_100)
#choice_check_100.to_csv('instances/full_instances/full_100.csv')
frequency_collection_100 = frequency_collection_100.astype(int)
frequency_check_100 = pd.DataFrame(frequency_collection_100)
#frequency_check_100.to_csv('instances/full_instances/frequency_100.csv')

offer_times_100 = assortment_info_df_100['offer_times'].tolist()
print('the number of assortments with at least 100 purchases is',len(assortments_100))

the number of assortments with at least 100 purchases is 11


In [11]:
assortment_info_df_15 = assortment_info_df[(assortment_info_df['offer_times']>=15)]
assortments_15 = []
for i in range(len(assortment_info_df_15['assortments'])):
    assortments_15.append(assortment_info_df_15['assortments'][i])


jd_choice_15 = pd.DataFrame(columns=jd_choice.columns)

num = 0
for i in range(len(jd_choice)):
    if jd_choice['clickset'][i] in assortments_15:
        jd_choice_15.loc[num] = jd_choice.loc[i]
        num = num+1 

collection_15 = assortment_info_df_15['assortments']
choice_collection_15 = np.zeros((n,len(collection_15)))

frequency_collection_15 = np.zeros((n,len(collection_15)))

purchaseset_list_15 = jd_choice_15['purchaseset'].tolist()

inc_prod_num_15 = assortment_info_df_15['includ_prod_num']
choice_prob_15 = jd_choice_15['choice prob']
frequency_15 = jd_choice_15['purchasenum']

global_num = 0
for i in range(len(collection_15)):
    local_num =0
    while(local_num < inc_prod_num_15[i]):
        if purchaseset_list_15[global_num][0] in assortments_15[i]:
            choice_collection_15[purchaseset_list_15[global_num][0]][i] = choice_prob_15[global_num]
            frequency_collection_15[purchaseset_list_15[global_num][0]][i] = frequency_15[global_num]
            global_num = global_num +1
            local_num = local_num +1
        else:
            print(purchaseset_list_15[global_num])
            print(assortments_15[i])
            print('error!')
            break
#print(global_num)  

choice_check_15 = pd.DataFrame(choice_collection_15)
#choice_check_15.to_csv('instances/full_instances/full_15.csv')
frequency_collection_15 = frequency_collection_15.astype(int)
frequency_check_15 = pd.DataFrame(frequency_collection_15)
#frequency_check_15.to_csv('instances/full_instances/frequency_15.csv')

offer_times_15 = assortment_info_df_15['offer_times'].tolist()
print('the number of assortments with at least 15 purchases is',len(assortments_15))

the number of assortments with at least 15 purchases is 38


In [12]:
import pyomo.environ as pyo
from pyomo.environ import *

In [13]:
def mdm_kl_w_lam(lam,data,collection,offertimes):
    
    n, m = data.shape
    
    ind = [[] for i in range(n)]  ## keeps track of the assortments including product i

    for i in range(len(collection)):
        for j in collection[i]:
            ind[j].append(i)  ## assortment i includes product j
    
    model = pyo.ConcreteModel()
    model.x = pyo.Var(range(n), range(m), domain= pyo.NonNegativeReals)
    
    model.norm_constraint = pyo.ConstraintList()
    for i in range(len(collection)):
        model.norm_constraint.add(sum(model.x[k, i] for k in collection[i]) == 1)
        for j in range(n):
            if j not in collection[i]:
                model.norm_constraint.add(model.x[j, i] == 0)
    
    model.mdm_constraint = pyo.ConstraintList()
    for i in range(n):
        for j in range(len(ind[i]) - 1):
            for k in range(j + 1, len(ind[i])):
                if lam[ind[i][j]] >= lam[ind[i][k]]:
                    model.mdm_constraint.add(model.x[i, ind[i][k]] - model.x[i, ind[i][j]]  >= 0)
                if lam[ind[i][j]] <= lam[ind[i][k]]:
                    model.mdm_constraint.add(model.x[i, ind[i][j]] - model.x[i, ind[i][k]]  >= 0)
    
    model.obj = pyo.Objective(expr= -sum( sum(offertimes[j] *data[i, j] * pyo.log(model.x[i, j]) for i in collection[j]) for j in range(m))/np.sum(offertimes),sense=pyo.minimize)
    
    solver = pyo.SolverFactory('ipopt')  # You can change the solver as needed
    solver.options['max_iter'] = 10000
    
    try:
        # Your optimization solve code here
        results = solver.solve(model,tee=True)
        status = str(results.solver.status)
        # Access the optimal solution
        if status == pyo.SolverStatus.ok:
            print("Solver found a solution.")
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            return [pyo.value(model.obj), x_values, results.solver.time]
        
        elif results.solver.status == SolverStatus.warning:
            print("Solver issued a warning.")
            x_values = [[model.x[i, j].value for j in range(m)] for i in range(n)]
            x_values = np.array(x_values)
            
            return [pyo.value(model.obj), x_values, results.solver.time]
        # Check if the solver status is an error
        if results.solver.status != pyo.SolverStatus.ok:
            raise ValueError(f"Solver failed with status: {results.solver.status}")
    
    except ValueError as e:
        print(f"Error: {e}")
        
    

In [ ]:
df_lam20 = pd.read_csv('limit/probability/mdm/lambda/lam20.csv')
df_lam20 = df_lam20.drop(df_lam20.columns[0],axis=1)
df_lam20 = df_lam20.values.tolist()
lam20 = []
for i in range(len(df_lam20)):
    lam20.append(df_lam20[i][0])
lam20
result_20 = mdm_kl_w_lam(lam20,choice_collection_20,collection_20,offer_times_20)

mdm_kl_20 = 0 
for i in range(result_20[1].shape[0]):
    for j in range(result_20[1].shape[1]):
        if result_20[1][i][j] >0:
            mdm_kl_20 = mdm_kl_20 -offer_times_20[j]*choice_collection_20[i][j]*np.log(result_20[1][i][j])/np.sum(offer_times_20)
mdm_kl_20

In [31]:
df_lam30 = pd.read_csv('limit/probability/mdm/lambda/lam30.csv')
df_lam30 = df_lam30.drop(df_lam30.columns[0],axis=1)
df_lam30 = df_lam30.values.tolist()
lam30 = []
for i in range(len(df_lam30)):
    lam30.append(df_lam30[i][0])

result_30 = mdm_kl_w_lam(lam30,choice_collection_30,collection_30,offer_times_30)
result_30


Ipopt 3.12.12: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      216
Number of nonzeros in inequality constraint Jacobian.:      740
Number of nonzeros in Lagrangian Hessian.............:       63

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Total number of variables..................

[0.5830423281497239,
 array([[0.74235627, 0.73759203, 0.65833075, 0.77333664, 0.91441959,
         0.74235762, 0.74242434, 0.65833006, 0.74235342, 0.74235565,
         0.74234725, 0.74234807, 0.70421801, 0.65837345, 0.65833006,
         0.65832517, 0.65832606, 0.75670837, 0.74235192, 0.7375906 ,
         0.70421749, 0.70768945, 0.70418963, 0.70421878],
        [0.25764373, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.04770837, 0.17836922, 0.19588773,
         0.05699818, 0.141768  , 0.04930492, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.26240797, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.1652036 , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.

In [32]:
df_lam40 = pd.read_csv('limit/probability/mdm/lambda/lam40.csv')
df_lam40 = df_lam40.drop(df_lam40.columns[0],axis=1)
df_lam40 = df_lam40.values.tolist()
lam40 = []
for i in range(len(df_lam40)):
    lam40.append(df_lam40[i][0])

result_40 = mdm_kl_w_lam(lam40,choice_collection_40,collection_40,offer_times_40)
result_40


Ipopt 3.12.12: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      171
Number of nonzeros in inequality constraint Jacobian.:      446
Number of nonzeros in Lagrangian Hessian.............:       48

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Total number of variables..................

[0.5791111643477619,
 array([[0.74111335, 0.71705989, 0.65718473, 0.77333558, 0.91441949,
         0.74111529, 0.74181634, 0.657184  , 0.74110947, 0.74111263,
         0.74110327, 0.65719211, 0.657184  , 0.65717898, 0.65717985,
         0.74110759, 0.71704731, 0.71704794, 0.71704826],
        [0.25888665, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.04675044, 0.17922842, 0.1880558 ,
         0.04677276, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.28294011, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.1599776 , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.34281527, 0.        , 0.        ,
         0.        , 0.        , 0.29606556, 0.        , 0.        ,
         0.        , 0.        , 0.29606556, 0

In [33]:
df_lam50 = pd.read_csv('limit/probability/mdm/lambda/lam50.csv')
df_lam50 = df_lam50.drop(df_lam50.columns[0],axis=1)
df_lam50 = df_lam50.values.tolist()
lam50 = []
for i in range(len(df_lam50)):
    lam50.append(df_lam50[i][0])

result_50 = mdm_kl_w_lam(lam50,choice_collection_50,collection_50,offer_times_50)
result_50


Ipopt 3.12.12: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      135
Number of nonzeros in inequality constraint Jacobian.:      260
Number of nonzeros in Lagrangian Hessian.............:       37

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Total number of variables..................

[0.577626203363289,
 array([[7.36985801e-01, 7.16675612e-01, 6.56185233e-01, 7.73333921e-01,
         9.14419595e-01, 7.38498769e-01, 7.41760776e-01, 6.56185080e-01,
         7.36984322e-01, 7.36981406e-01, 6.56187250e-01, 6.56184714e-01,
         7.16671022e-01, 7.16671222e-01, 7.16671334e-01],
        [2.63014199e-01, 5.00504491e-41, 5.00504491e-41, 5.00504491e-41,
         5.00504491e-41, 5.00504491e-41, 5.00504491e-41, 5.64834777e-02,
         1.82087804e-01, 5.64836685e-02, 5.00504491e-41, 5.00504491e-41,
         5.00504491e-41, 5.00504491e-41, 5.00504491e-41],
        [5.00504491e-41, 2.83324388e-01, 5.00504491e-41, 5.00504491e-41,
         5.00504491e-41, 5.00504491e-41, 5.00504491e-41, 5.00504491e-41,
         5.00504491e-41, 5.00504491e-41, 1.60446124e-01, 5.00504491e-41,
         5.00504491e-41, 5.00504491e-41, 5.00504491e-41],
        [5.00504491e-41, 5.00504491e-41, 3.43814767e-01, 5.00504491e-41,
         5.00504491e-41, 5.00504491e-41, 5.00504491e-41, 2.87331443e-01,
   

In [35]:
df_lam60 = pd.read_csv('limit/probability/mdm/lambda/lam60.csv')
df_lam60 = df_lam60.drop(df_lam60.columns[0],axis=1)
df_lam60 = df_lam60.values.tolist()
lam60 = []
for i in range(len(df_lam60)):
    lam60.append(df_lam60[i][0])

result_60 = mdm_kl_w_lam(lam60,choice_collection_60,collection_60,offer_times_60)
result_60

Ipopt 3.12.12: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      117
Number of nonzeros in inequality constraint Jacobian.:      190
Number of nonzeros in Lagrangian Hessian.............:       31

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Total number of variables..................

[0.5751299252119062,
 array([[0.73899503, 0.71437513, 0.65257729, 0.77333473, 0.9144195 ,
         0.73900393, 0.74176679, 0.73899404, 0.6525846 , 0.65257675,
         0.7143594 , 0.71436017, 0.7143605 ],
        [0.26100497, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.0491011 , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.28562487, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.1621276 , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.34742271, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.25808601,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.22666527, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.

In [36]:
df_lam75 = pd.read_csv('limit/probability/mdm/lambda/lam75.csv')
df_lam75 = df_lam75.drop(df_lam75.columns[0],axis=1)
df_lam75 = df_lam75.values.tolist()
lam75 = []
for i in range(len(df_lam75)):
    lam75.append(df_lam75[i][0])

result_75 = mdm_kl_w_lam(lam75,choice_collection_75,collection_75,offer_times_75)
result_75

Ipopt 3.12.12: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      108
Number of nonzeros in inequality constraint Jacobian.:      154
Number of nonzeros in Lagrangian Hessian.............:       28

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Total number of variables..................

[0.5731583014460984,
 array([[0.74200596, 0.70552601, 0.65316359, 0.77333461, 0.91441966,
         0.74200691, 0.74208554, 0.74200561, 0.65319302, 0.65316307,
         0.70519   , 0.70519041],
        [0.25799404, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.07568415, 0.        , 0.        ,
         0.        , 0.        ],
        [0.        , 0.29447399, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.16449893, 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.34683641, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.25765039,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.22666539, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.08558034,
         0.        , 0.        , 0.        , 0.   

In [37]:
df_lam100 = pd.read_csv('limit/probability/mdm/lambda/lam100.csv')
df_lam100 = df_lam100.drop(df_lam100.columns[0],axis=1)
df_lam100 = df_lam100.values.tolist()
lam100 = []
for i in range(len(df_lam100)):
    lam100.append(df_lam100[i][0])

result_100 = mdm_kl_w_lam(lam100,choice_collection_100,collection_100,offer_times_100)
result_100


Ipopt 3.12.12: max_iter=10000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       99
Number of nonzeros in inequality constraint Jacobian.:      130
Number of nonzeros in Lagrangian Hessian.............:       26

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Total number of variables..................

[0.5727838904338546,
 array([[0.74200686, 0.65316382, 0.7733345 , 0.91441965, 0.7420078 ,
         0.74208021, 0.74200651, 0.65319295, 0.65316331, 0.70523318,
         0.70523359],
        [0.25799314, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.07568348, 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.16449922, 0.        , 0.        ,
         0.        ],
        [0.        , 0.34683618, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.25765021, 0.        ,
         0.        ],
        [0.        , 0.        , 0.2266655 , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , 0.08558035, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.0707438 ,
         0.        ],
        [0.        

In [46]:
dt_20 = 0
for i in range(choice_collection_20.shape[0]):
    for j in range(choice_collection_20.shape[1]):
        if choice_collection_20[i][j] >0:
            dt_20 = dt_20 + offer_times_20[j]*choice_collection_20[i][j]*np.log(choice_collection_20[i][j])/np.sum(offer_times_20)
dt_20 + result_20[0]


0.002365561927539339

In [47]:
dt_30 = 0
for i in range(choice_collection_30.shape[0]):
    for j in range(choice_collection_30.shape[1]):
        if choice_collection_30[i][j] >0:
            dt_30 = dt_30 + offer_times_30[j]*choice_collection_30[i][j]*np.log(choice_collection_30[i][j])/np.sum(offer_times_30)
dt_30 +result_30[0]

0.0022146226997870055

In [48]:
dt_40 = 0
for i in range(choice_collection_40.shape[0]):
    for j in range(choice_collection_40.shape[1]):
        if choice_collection_40[i][j] >0:
            dt_40 = dt_40 + offer_times_40[j]*choice_collection_40[i][j]*np.log(choice_collection_40[i][j])/np.sum(offer_times_40)
dt_40 +result_40[0]

0.0021379140023223275

In [49]:
dt_50 = 0
for i in range(choice_collection_50.shape[0]):
    for j in range(choice_collection_50.shape[1]):
        if choice_collection_50[i][j] >0:
            dt_50 = dt_50 + offer_times_50[j]*choice_collection_50[i][j]*np.log(choice_collection_50[i][j])/np.sum(offer_times_50)
dt_50 +result_50[0]

0.0014255202676115575

In [50]:
dt_60 = 0
for i in range(choice_collection_60.shape[0]):
    for j in range(choice_collection_60.shape[1]):
        if choice_collection_60[i][j] >0:
            dt_60 = dt_60 + offer_times_60[j]*choice_collection_60[i][j]*np.log(choice_collection_60[i][j])/np.sum(offer_times_60)
dt_60 +result_60[0]

0.0012301082046082268

In [51]:
dt_75 = 0
for i in range(choice_collection_75.shape[0]):
    for j in range(choice_collection_75.shape[1]):
        if choice_collection_75[i][j] >0:
            dt_75 = dt_75 + offer_times_75[j]*choice_collection_75[i][j]*np.log(choice_collection_75[i][j])/np.sum(offer_times_75)
dt_75 +result_75[0]

0.0009438850108706154

In [52]:
dt_100 = 0
for i in range(choice_collection_100.shape[0]):
    for j in range(choice_collection_100.shape[1]):
        if choice_collection_100[i][j] >0:
            dt_100 = dt_100 + offer_times_100[j]*choice_collection_100[i][j]*np.log(choice_collection_100[i][j])/np.sum(offer_times_100)
dt_100 +result_100[0]

0.0009544922840073466

In [63]:
offertime_list = [20,30,40,50,60,75,100]
deterministic_terms_lst = [dt_20,dt_30,dt_40,dt_50,dt_60,dt_75,dt_100]
mdm_kl_lst = [result_20[0],result_30[0],result_40[0],result_50[0],result_60[0],result_75[0],result_100[0]]
mdm_kl_prob_lst = [result_20[1],result_30[1],result_40[1],result_50[1],result_60[1],result_75[1],result_100[1]]
mdm_2stage_time_lst = [result_20[-1],result_30[-1],result_40[-1],result_50[-1],result_60[-1],result_75[-1],result_100[-1]]

In [55]:
offertime_lst = [offer_times_20,offer_times_30,offer_times_40,offer_times_50,offer_times_60,offer_times_75,offer_times_100]
choice_collection_lst = [choice_collection_20,choice_collection_30,choice_collection_40,choice_collection_50,choice_collection_60,choice_collection_75,choice_collection_100]

In [56]:
lcmnl_kl_lst = []
for k in range(len(offertime_list)):
    df_lcmnl_prob = pd.read_csv('limit/probability/lcmnl_em/prob_offertimes'+str(offertime_list[k])+'.csv')
    df_lcmnl_prob = df_lcmnl_prob.drop(df_lcmnl_prob.columns[0],axis=1)
    lcmnl_prob = np.array(df_lcmnl_prob)
    lcmnl_kl = 0 
    for i in range(lcmnl_prob.shape[0]):
        for j in range(lcmnl_prob.shape[1]):
            if lcmnl_prob[i][j] >0:
                lcmnl_kl = lcmnl_kl-offertime_lst[k][j]*choice_collection_lst[k][i][j]*np.log(lcmnl_prob[i][j])/np.sum(offertime_lst[k])
    lcmnl_kl_lst.append(lcmnl_kl)

In [57]:
lcmnl_kl_lst

[0.5856461268848074,
 0.5824346005751264,
 0.5784750511347461,
 0.5773594592095833,
 0.5748309570377605,
 0.5731514699887693,
 0.572776981343711]

In [59]:
mnl_kl_lst = []
for k in range(len(offertime_list)):
    df_mnl_prob = pd.read_csv('limit/probability/mnl/prob_offertimes'+str(offertime_list[k])+'.csv')
    df_mnl_prob = df_mnl_prob.drop(df_mnl_prob.columns[0],axis=1)
    mnl_prob = np.array(df_mnl_prob)
    mnl_kl = 0 
    for i in range(mnl_prob.shape[0]):
        for j in range(mnl_prob.shape[1]):
            if mnl_prob[i][j] >0:
                mnl_kl = mnl_kl-offertime_lst[k][j]*choice_collection_lst[k][i][j]*np.log(mnl_prob[i][j])/np.sum(offertime_lst[k])
    mnl_kl_lst.append(mnl_kl)

In [60]:
mnl_kl_lst

[0.5937364984479344,
 0.5896719056441664,
 0.585088075111838,
 0.5822274721946021,
 0.5789814998084744,
 0.5772794827261699,
 0.576775281965606]

In [61]:
kl_df = pd.DataFrame({'mdm':mdm_kl_lst,'lcmnl':lcmnl_kl_lst,'mnl':mnl_kl_lst})
kl_df

,mdm,lcmnl,mnl
0,0.586247,0.585646,0.593736
1,0.583042,0.582435,0.589672
2,0.579111,0.578475,0.585088
3,0.577626,0.577359,0.582227
4,0.575130,0.574831,0.578981
5,0.573158,0.573151,0.577279
6,0.572784,0.572777,0.576775


In [65]:
offertime_list

[20, 30, 40, 50, 60, 75, 100]

In [ ]:
mdm_kl_lst = []
for k in range(len(offertime_list)):
    df_mdm_prob = pd.read_csv('limit/probability/mdm/prob_offertimes'+str(offertime_list[k])+'.csv')
    df_mdm_prob = df_mdm_prob.drop(df_mdm_prob.columns[0],axis=1)
    mdm_prob = np.array(df_mdm_prob)
    mdm_kl = 0 
    for i in range(mdm_prob.shape[0]):
        for j in range(mdm_prob.shape[1]):
            if mdm_prob[i][j] >0:
                mdm_kl = mdm_kl-offertime_lst[k][j]*choice_collection_lst[k][i][j]*np.log(mdm_prob[i][j])/np.sum(offertime_lst[k])
    mdm_kl_lst.append(mdm_kl)

In [66]:
mdm_kl_loss_lst = []
lcmnl_kl_loss_lst = []
mnl_kl_loss_lst = []
for i in range(len(offertime_list)):
    mdm_kl_loss_lst.append(mdm_kl_lst[i]+deterministic_terms_lst[i])
    lcmnl_kl_loss_lst.append(lcmnl_kl_lst[i]+deterministic_terms_lst[i])
    mnl_kl_loss_lst.append(mnl_kl_lst[i]+deterministic_terms_lst[i])
kl_loss_df = pd.DataFrame({'mdm':mdm_kl_loss_lst,'lcmnl':lcmnl_kl_loss_lst,'mnl':mnl_kl_loss_lst})
kl_loss_df

,mdm,lcmnl,mnl
0,0.002366,0.001765,0.009855
1,0.002215,0.001607,0.008844
2,0.002138,0.001502,0.008115
3,0.001426,0.001159,0.006027
4,0.001230,0.000931,0.005082
5,0.000944,0.000937,0.005065
6,0.000954,0.000948,0.004946


In [67]:
summary = pd.read_csv('limit/avg_loss_summary.csv')
summary


,Unnamed: 0,avg_mdm_loss,mdm_time,avg_rum_loss,rum_runtime,avg_mem_loss,mem_time,avg_mnl_loss,mnl_time,avg_mccm_loss,mccm_runtime,avg_lcmnl_loss,lcmnl_runtime,avg_indep_loss,indep_runtime
0,0,0.008040,16.726812,0.007411,126.973082,0.017497,0.307991,0.033804,0.228111,0.028520,30.130095,0.013234,75.667320,0.537818,0.001921
1,1,0.007380,6.037370,0.006852,107.641489,0.016652,0.347865,0.031831,0.236472,0.026237,17.218676,0.011008,80.473350,0.533980,0.001474
2,2,0.006772,1.644473,0.006434,88.861326,0.015785,0.362538,0.029103,0.259305,0.022564,7.526283,0.011670,94.031553,0.528506,0.001333
3,3,0.005148,0.588080,0.004752,74.893178,0.013033,0.305427,0.023923,0.252010,0.011679,2.118375,0.009240,67.067113,0.520491,0.000895
4,4,0.004419,0.229502,0.004187,64.534657,0.011312,0.295169,0.020142,0.236779,0.009621,1.742772,0.007905,71.974360,0.516003,0.000533
5,5,0.004288,0.227669,0.004205,62.552825,0.010772,0.306293,0.020215,0.237868,0.008253,1.729044,0.007978,80.520300,0.514184,0.000569
6,6,0.004336,0.184779,0.004253,56.296840,0.010789,0.520295,0.019391,0.478462,0.007264,2.049280,0.008070,68.130978,0.515352,0.000699


In [90]:
collection_lst = [collection_20,collection_30,collection_40,collection_50,collection_60,collection_75,collection_100]
idep_model_kl_lst = []
for k in range(len(offertime_list)):
    
    idep_model_kl = 0 
    for i in range(len(collection_lst[k])):
        assortment = collection_lst[k][i]
        for j in assortment:
            idep_model_kl = idep_model_kl - offertime_lst[k][i]*choice_collection_lst[k][j][i] * np.log(1/len(assortment))/np.sum(offertime_lst[k])
    idep_model_kl_lst.append(idep_model_kl)

idep_model_kl_loss_lst = []
for i in range(len(idep_model_kl_lst)):
    idep_model_kl_loss_lst.append(idep_model_kl_lst[i]+deterministic_terms_lst[i])
idep_model_kl_loss_lst

[0.18114384736876554,
 0.17780567134583325,
 0.17390079250187884,
 0.16640308992753106,
 0.16327485128706964,
 0.1615081748006877,
 0.16234942517958517]

In [72]:
mdm_1stage_time_lst = summary['mdm_time'].tolist()
mdm_kl_time_lst = []
for i in range(len(mdm_1stage_time_lst)):
    mdm_kl_time_lst.append(mdm_1stage_time_lst[i] + mdm_2stage_time_lst[i])
mdm_kl_time_lst

[17.193889141082764,
 6.611323833465575,
 2.078651189804077,
 0.9362821578979492,
 0.6637389659881592,
 0.5345120429992676,
 0.48494791984558105]

In [74]:
lcmnl_kl_time_lst = summary['lcmnl_runtime'].tolist()
mnl_kl_time_lst = summary['mnl_time'].tolist()

In [91]:
kl_summary = pd.DataFrame({'offertimes':offertime_list,'mdm_kl':mdm_kl_loss_lst,'mdm_time':mdm_kl_time_lst,
                           'lcmnl_kl':lcmnl_kl_loss_lst,'lcmnl_time':lcmnl_kl_time_lst,
                           'mnl_kl':mnl_kl_loss_lst,'mnl_time':mnl_kl_time_lst,
                           'indep_kl_loss':idep_model_kl_loss_lst})
kl_summary.to_csv('limit/kl_summary.csv')
kl_summary

,offertimes,mdm_kl,mdm_time,lcmnl_kl,lcmnl_time,mnl_kl,mnl_time,indep_kl_loss
0,20,0.002366,17.193889,0.001765,75.667320,0.009855,0.228111,0.181144
1,30,0.002215,6.611324,0.001607,80.473350,0.008844,0.236472,0.177806
2,40,0.002138,2.078651,0.001502,94.031553,0.008115,0.259305,0.173901
3,50,0.001426,0.936282,0.001159,67.067113,0.006027,0.252010,0.166403
4,60,0.001230,0.663739,0.000931,71.974360,0.005082,0.236779,0.163275
5,75,0.000944,0.534512,0.000937,80.520300,0.005065,0.237868,0.161508
6,100,0.000954,0.484948,0.000948,68.130978,0.004946,0.478462,0.162349


In [17]:
df_lcmnl20_prob = pd.read_csv('limit/probability/lcmnl_em/prob_offertimes20.csv')
df_lcmnl20_prob = df_lcmnl20_prob.drop(df_lcmnl20_prob.columns[0],axis=1)
lcmnl20_prob = np.array(df_lcmnl20_prob)
lcmnl_kl_20 = 0 
for i in range(lcmnl20_prob.shape[0]):
    for j in range(lcmnl20_prob.shape[1]):
        if lcmnl20_prob[i][j] >0:
            lcmnl_kl_20 = lcmnl_kl_20-offer_times_20[j]*choice_collection_20[i][j]*np.log(lcmnl20_prob[i][j])/np.sum(offer_times_20)
lcmnl_kl_20
    

In [27]:
df_mnl20_prob = pd.read_csv('limit/probability/mnl/prob_offertimes20.csv')
df_mnl20_prob = df_mnl20_prob.drop(df_mnl20_prob.columns[0],axis=1)
mnl20_prob = np.array(df_mnl20_prob)
mnl_kl_20 = 0 
for i in range(mnl20_prob.shape[0]):
    for j in range(mnl20_prob.shape[1]):
        if mnl20_prob[i][j] >0:
            mnl_kl_20 = mnl_kl_20 -offer_times_20[j]*choice_collection_20[i][j]*np.log(mnl20_prob[i][j])/np.sum(offer_times_20)
mnl_kl_20

0.5937364984479344